# Initialize

## Imports

In [ ]:
import json
import flask 
import locale

# Variables & Startup

In [ ]:
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')

In [ ]:
with open ("./../data/books.json", 'rb') as fp:
    books = json.load(fp)

In [ ]:
app = flask.Flask('my app')

### sample of single book data

In [ ]:
print(len(books))
print(books[0]['amazon_avg'])
print(books[0]['amazon_num'])

# Functions

In [ ]:
def get_all_categories(books):
    categories = []
    categories.append('All')
    for book in books:
        category = book['category']
        if category not in categories:
            categories.append(category)
    
    categories.sort()
    return categories    

In [ ]:
def get_total_books(categories):
    total = 0
    for category in categories:
        total = total + categories[category]

    return total

In [ ]:
def create_category_data_structure(categories, books):
    category_ds = []
    for category in categories:
        books_all = 0
        books_en  = 0
        books_de  = 0
        
        for book in books:
            if (book['category'] == category) or (category == 'All'):
                if book['language'] == 'en':
                    books_all += 1
                    books_en  += 1
                if book['language'] == 'de':
                    books_all += 1
                    books_de  += 1
        
        url_slug = category.lower().replace("&","-").replace(" ","")
        
        category_ds.append({'name': category, 
                            'books_all': books_all, 'books_en': books_en, 'books_de': books_de, 
                            'url_all': url_slug + "-all", 'url_en': url_slug + "-en", 'url_de': url_slug + "-de",})
    
    return category_ds

# Main

In [ ]:
categories = get_all_categories(books)
categories[0:2]

In [ ]:
category_ds = create_category_data_structure(categories, books)
category_ds[0:2]

## Create overview (index.html)

In [ ]:
with app.app_context():
    rendered = flask.render_template('categories.html', categories=category_ds)

with open("./../html-output/!index.html", 'w', encoding="utf-8") as file:
    file.write(rendered)
    file.close

## Create category htmls

In [ ]:
languages = ['all', 'en', 'de']

In [ ]:
for category in category_ds:
    for language in languages:
        books_in_category = [ x for x in books if ((x['category'] == category['name']) and ((x['language'] == language) or (language == 'all'))) ]
        books_in_category.sort(key=lambda k: k['amazon_num'], reverse=True)
    
    
    
        with app.app_context():
            rendered = flask.render_template('books.html', title=category['name'], books=books_in_category)

        with open("./../html-output/" + category[('url_'+ language)] + ".html", 'w', encoding="utf-8") as file:
            file.write(rendered)
            file.close